In [3]:
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [4]:
test=pd.read_excel(r"../data/chem_test.xlsx")

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
gpr_nox=torch.load("../models/nox_seasonality",weights_only=False,map_location='cuda')

In [7]:
gpr_no2=torch.load("../models/no2_seasonality",weights_only=False,map_location='cuda')

In [8]:
test=test.loc[(test.corrected_week>0)&(test.corrected_week<52.5),:]

In [9]:
def prediction(test,particle,model):
    gpr_points=model(torch.tensor(test.corrected_week).float().to(device))
    test["scale"]=gpr_points[0].cpu().detach().numpy()
    test_3=test.set_index("cpnr").loc[test.groupby(test.cpnr).maalnr.count()>=3,:]
    BC_1_raw=test_3.loc[test_3.maalnr==1,particle].groupby("cpnr").first()
    BC_1_scale=test_3.loc[test_3.maalnr==1,"scale"].groupby("cpnr").first()
    BC_2_raw=test_3.loc[test_3.maalnr==2,particle].groupby("cpnr").first()
    BC_2_scale=test_3.loc[test_3.maalnr==2,"scale"].groupby("cpnr").first()
    BC_3_scale=test_3.loc[test_3.maalnr==3,"scale"].groupby("cpnr").first()
    BC_3_raw=test_3.loc[test_3.maalnr==3,particle].groupby("cpnr").first()
    true_3=(BC_2_raw*BC_3_scale/BC_2_scale+BC_1_raw*BC_3_scale/BC_1_scale)/2
    true_1=(BC_2_raw*BC_1_scale/BC_2_scale+BC_3_raw*BC_1_scale/BC_3_scale)/2
    true_2=(BC_1_raw*BC_2_scale/BC_1_scale+BC_3_raw*BC_2_scale/BC_3_scale)/2
    prediction_matrix=pd.DataFrame([true_1,BC_1_raw,BC_1_scale,true_2,BC_2_raw,BC_2_scale,true_3,BC_3_raw,BC_3_scale],index=["predict_1","raw_1","scale_1","predict_2","raw_2","scale_2","predict_3","raw_3","scale_3"]).T
    return prediction_matrix

In [12]:
prediction_matrix_nox=prediction(test,"nox",gpr_nox).dropna()
prediction_matrix_no2=prediction(test,"no2",gpr_no2).dropna()

In [13]:
nox_r2=r2_score(prediction_matrix_nox.raw_1,prediction_matrix_nox.predict_1),r2_score(prediction_matrix_nox.raw_2,prediction_matrix_nox.predict_2),r2_score(prediction_matrix_nox.raw_3,prediction_matrix_nox.predict_3)

In [39]:
no2_r2_mean=r2_score(prediction_matrix_no2.raw_1,mean_no2_1),r2_score(prediction_matrix_no2.raw_2,mean_no2_2),r2_score(prediction_matrix_no2.raw_3,mean_no2_3)

In [44]:
no2_r2_mean=r2_score(mean_1,prediction_matrix_no2.raw_1),r2_score(mean_2,prediction_matrix_no2.raw_2),r2_score(mean_no2,prediction_matrix_no2.raw_3)

In [45]:
nox_r2_mean=r2_score(prediction_matrix_nox.raw_1,mean_1),r2_score(prediction_matrix_nox.raw_2,mean_2),r2_score(prediction_matrix_nox.raw_3,mean_3)

In [46]:
np.mean(nox_r2_mean),np.mean(nox_r2)

(-0.3819490309478466, 0.2884927836943007)

In [47]:
np.mean(no2_r2_mean),np.mean(no2_r2)

(0.249752599453953, 0.4168952765224509)

In [49]:
nox_r2_mean

(0.27641692080265845, -1.6122736928840808, 0.1900096792378828)

In [48]:
no2_r2

(0.5758409768987418, 0.20088129937141996, 0.47396355329719086)

In [ ]:
no2_r2=r2_score(prediction_matrix_no2.raw_1,prediction_matrix_no2.predict_1),r2_score(prediction_matrix_no2.raw_2,prediction_matrix_no2.predict_2),r2_score(prediction_matrix_no2.raw_3,prediction_matrix_no2.predict_3)

In [ ]:
no2_r2=r2_score(prediction_matrix_no2.raw_1,prediction_matrix_no2.predict_1),r2_score(prediction_matrix_no2.raw_2,prediction_matrix_no2.predict_2),r2_score(prediction_matrix_no2.raw_3,prediction_matrix_no2.predict_3)

In [36]:
mean_3=prediction_matrix_nox.loc[:,["raw_1","raw_2"]].mean(axis=1)
mean_2=prediction_matrix_nox.loc[:,["raw_1","raw_3"]].mean(axis=1)
mean_1=prediction_matrix_nox.loc[:,["raw_2","raw_3"]].mean(axis=1)
mean_no2_3=prediction_matrix_no2.loc[:,["raw_1","raw_2"]].mean(axis=1)
mean_no2_2=prediction_matrix_no2.loc[:,["raw_1","raw_3"]].mean(axis=1)
mean_no2_1=prediction_matrix_no2.loc[:,["raw_2","raw_3"]].mean(axis=1)

In [15]:
np.mean(nox_r2),np.mean(no2_r2)

(0.2884927836943007, 0.4168952765224509)

In [99]:
np.mean([0.5758409768987418, 0.20088129937141996, 0.47396355329719086])

0.4168952765224509

In [100]:
prediction_matrix_no2.corr()

,predict_1,raw_1,scale_1,predict_2,raw_2,scale_2,predict_3,raw_3,scale_3
predict_1,1.000000,0.769180,0.033110,0.876078,0.822120,-0.106590,0.899019,0.907773,-0.004605
raw_1,0.769180,1.000000,0.122523,0.934115,0.548399,-0.203638,0.924231,0.749171,0.047017
scale_1,0.033110,0.122523,1.000000,-0.024061,-0.291323,-0.557803,0.001434,0.120630,0.719101
predict_2,0.876078,0.934115,-0.024061,1.000000,0.630141,-0.026401,0.906721,0.896213,-0.069923
raw_2,0.822120,0.548399,-0.291323,0.630141,1.000000,0.167243,0.813417,0.536777,-0.176907
scale_2,-0.106590,-0.203638,-0.557803,-0.026401,0.167243,1.000000,-0.115088,-0.124269,-0.315134
predict_3,0.899019,0.924231,0.001434,0.906721,0.813417,-0.115088,1.000000,0.759926,0.038759
raw_3,0.907773,0.749171,0.120630,0.896213,0.536777,-0.124269,0.759926,1.000000,0.069670
scale_3,-0.004605,0.047017,0.719101,-0.069923,-0.176907,-0.315134,0.038759,0.069670,1.000000


In [101]:
prediction_matrix_nox.corr()

,predict_1,raw_1,scale_1,predict_2,raw_2,scale_2,predict_3,raw_3,scale_3
predict_1,1.000000,0.589576,0.526829,0.083466,0.071326,-0.300363,0.671152,0.804646,0.306662
raw_1,0.589576,1.000000,0.500013,0.075557,-0.070042,-0.282069,0.846295,0.706794,0.349445
scale_1,0.526829,0.500013,1.000000,-0.376510,-0.357020,-0.709868,0.459632,0.519601,0.627283
predict_2,0.083466,0.075557,-0.376510,1.000000,0.500378,0.752163,-0.042992,-0.033858,-0.455345
raw_2,0.071326,-0.070042,-0.357020,0.500378,1.000000,0.612609,-0.032412,-0.160012,-0.403626
scale_2,-0.300363,-0.282069,-0.709868,0.752163,0.612609,1.000000,-0.378896,-0.408928,-0.666939
predict_3,0.671152,0.846295,0.459632,-0.042992,-0.032412,-0.378896,1.000000,0.841263,0.619915
raw_3,0.804646,0.706794,0.519601,-0.033858,-0.160012,-0.408928,0.841263,1.000000,0.604632
scale_3,0.306662,0.349445,0.627283,-0.455345,-0.403626,-0.666939,0.619915,0.604632,1.000000
